# Лабораторная работа №2: Исследование логистической и линейной регрессии

## 1. Загрузка и подготовка данных

In [1]:
import kagglehub
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, f1_score, mean_absolute_error, r2_score

# Загрузка и подготовка данных (повторяем шаги из лабы 1)

# Классификация
df_clf = pd.read_csv(kagglehub.dataset_download("ritesaluja/bank-note-authentication-uci-data") + "/BankNote_Authentication.csv")
X_clf = df_clf.drop('class', axis=1)
y_clf = df_clf['class']
X_clf_train, X_clf_test, y_clf_train, y_clf_test = train_test_split(
    X_clf, y_clf, test_size=0.2, random_state=42)

# Регрессия
df_reg = pd.read_csv(kagglehub.dataset_download("mirichoi0218/insurance") + "/insurance.csv")
X_reg_full = df_reg.drop('charges', axis=1)
y_reg_full = df_reg['charges']
X_reg_full_train, X_reg_full_test, y_reg_full_train, y_reg_full_test = train_test_split(
    X_reg_full, y_reg_full, test_size=0.2, random_state=42)
X_reg = df_reg.drop(['sex', 'smoker', 'region', 'charges'], axis=1)
y_reg = df_reg['charges']
X_reg_train, X_reg_test, y_reg_train, y_reg_test = train_test_split(
    X_reg, y_reg, test_size=0.2, random_state=42)

100%|██████████| 19.2k/19.2k [00:00<00:00, 8.07MB/s]

Extracting files...


100%|██████████| 16.0k/16.0k [00:00<00:00, 17.5MB/s]

Extracting files...


## 2. Создание бейзлайна и оценка качества

**Обучение модели**

In [2]:
from sklearn.linear_model import LogisticRegression, LinearRegression

# ЗАДАЧА КЛАССИФИКАЦИИ
log_reg = LogisticRegression(random_state=42, max_iter=1000)
log_reg.fit(X_clf_train, y_clf_train)

# ЗАДАЧА РЕГРЕССИИ
lin_reg = LinearRegression()
lin_reg.fit(X_reg_train, y_reg_train)

LinearRegression()

**Делаем предсказания и считаем метрики**

In [3]:
# КЛАССИФИКАЦИЯ
y_clf_pred_lr = log_reg.predict(X_clf_test)
acc_baseline_lr = accuracy_score(y_clf_test, y_clf_pred_lr)
f1_baseline_lr = f1_score(y_clf_test, y_clf_pred_lr)

print("Классификация (Бейзлайн)")
print(f"Accuracy: {acc_baseline_lr:.4f}")
print(f"F1-score: {f1_baseline_lr:.4f}")

# РЕГРЕССИЯ
y_reg_pred_lr = lin_reg.predict(X_reg_test)
mae_baseline_lr = mean_absolute_error(y_reg_test, y_reg_pred_lr)
r2_baseline_lr = r2_score(y_reg_test, y_reg_pred_lr)

print("\nРегрессия (Бейзлайн)")
print(f"Mean Absolute Error: {mae_baseline_lr:.2f}")
print(f"R²: {r2_baseline_lr:.4f}")

Классификация (Бейзлайн)
Accuracy: 0.9855
F1-score: 0.9843

Регрессия (Бейзлайн)
Mean Absolute Error: 9181.31
R²: 0.1549


Мы получили для классификации высокие результаты, а для регрессии - очень плохие.

## 3. Улучшение бейзлайна

**Гипотезы для улучшения**

1.  **Гипотеза 1 (Общая):** Как и для KNN, линейные модели чувствительны к масштабу признаков. Масштабирование должно улучшить сходимость и качество.
2.  **Гипотеза 2 (Общая):** Линейные модели могут переобучаться. Регуляризация (добавление "штрафа" за большие веса) может помочь. В `LogisticRegression` за это отвечает параметр `C`. Мы подберем оптимальное значение `C` через `GridSearchCV`. Для регрессии можно использовать `Ridge` регрессию (линейная регрессия с L2-регуляризацией), где подбирается параметр `alpha`.
3.  **Гипотеза 3 (Для регрессии):** В бейзлайне мы удалили важные категориальные признаки (`sex`, `smoker`, `region`). Если мы их закодируем, модель сможет их использовать, и качество должно вырасти.

Проверим их.

In [4]:
# --- ЗАДАЧА КЛАССИФИКАЦИИ (Гипотезы 1 и 2) ---

# Создаем пайплайн
pipeline_clf_lr = Pipeline([
    ('scaler', StandardScaler()),
    ('log_reg', LogisticRegression(random_state=42, max_iter=2000))
])

# Сетка параметров: перебираем разные значения для C
param_grid_clf_lr = {'log_reg__C': [0.001, 0.01, 0.1, 1, 10, 100]}

# Ищем лучшие параметр
grid_search_clf_lr = GridSearchCV(pipeline_clf_lr, param_grid_clf_lr, cv=5, scoring='f1')
grid_search_clf_lr.fit(X_clf_train, y_clf_train)

print(f"Лучший параметр C для классификации: {grid_search_clf_lr.best_params_['log_reg__C']}")
improved_model_clf_lr = grid_search_clf_lr

Лучший параметр C для классификации: 10


In [5]:
from sklearn.linear_model import Ridge

# ЗАДАЧА РЕГРЕССИИ (Гипотезы 1, 2, 3)

# 1. Используем тот же препроцессор, что и в 1 лабе
numeric_features = ['age', 'bmi', 'children']
categorical_features = ['sex', 'smoker', 'region']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(), categorical_features)])

# 2. Создаем пайплайн
pipeline_reg_lr = Pipeline([
    ('preprocessor', preprocessor),
    ('ridge', Ridge(random_state=42))
])

# 3. Сетка параметров: перебираем alpha
param_grid_reg_lr = {'ridge__alpha': [0.01, 0.1, 1, 10, 100]}

# 4. Ищем лучший параметр
grid_search_reg_lr = GridSearchCV(pipeline_reg_lr, param_grid_reg_lr, cv=5, scoring='r2')
grid_search_reg_lr.fit(X_reg_full_train, y_reg_full_train)

print(f"Лучший параметр alpha для регрессии: {grid_search_reg_lr.best_params_['ridge__alpha']}")
improved_model_reg_lr = grid_search_reg_lr

Лучший параметр alpha для регрессии: 1


**Делаем предсказания на улучшенных моделях и считаем метрики**

In [6]:
# Классификация
y_clf_pred_imp_lr = improved_model_clf_lr.predict(X_clf_test)
acc_improved_lr = accuracy_score(y_clf_test, y_clf_pred_imp_lr)
f1_improved_lr = f1_score(y_clf_test, y_clf_pred_imp_lr)

# Регрессия
y_reg_pred_imp_lr = improved_model_reg_lr.predict(X_reg_full_test)
mae_improved_lr = mean_absolute_error(y_reg_full_test, y_reg_pred_imp_lr)
r2_improved_lr = r2_score(y_reg_full_test, y_reg_pred_imp_lr)

# СРАВНЕНИЕ РЕЗУЛЬТАТОВ

print("Сравнение результатов")
print("Классификация:")
print(f"Accuracy: {acc_baseline_lr:.4f} -> {acc_improved_lr:.4f}")
print(f"F1-score: {f1_baseline_lr:.4f} -> {f1_improved_lr:.4f}")
print("\nРегрессия:")
print(f"MAE: {mae_baseline_lr:.2f} -> {mae_improved_lr:.2f}")
print(f"R²: {r2_baseline_lr:.4f} -> {r2_improved_lr:.4f}")

Сравнение результатов
Классификация:
Accuracy: 0.9855 -> 0.9855
F1-score: 0.9843 -> 0.9843

Регрессия:
MAE: 9181.31 -> 4186.91
R²: 0.1549 -> 0.7834


**Выводы по улучшению**

**Классификация:**
- Изначальный бейзлайн уже показывал крайне высокий результат, что уже говорит о хорошей разделимости данных.
- После масштабирования признаков и подбора оптимального `C` качество предсказаний не изменилось.

**Регрессия:**
- Бейзлайн был крайне слабым, так как не использовал категориальные признаки и не масштабировал числовые.
- После применения пайплайна с масштабированием числовых признаков, кодированием категориальных и применения регуляризации, качество модели выросло. R² стал довольно высоким, а MAE значительно снизилась.


Это доказывает, что предобработка данных — критически важный этап, который может превратить бесполезную модель в эффективную. Все гипотезы подтвердились.

## 4.	Имплементация алгоритма машинного обучения

In [9]:
# Базовый класс
class MyBaseRegression:
    def __init__(self, learning_rate=0.01, n_iterations=1000):
        self.lr = learning_rate
        self.n_iters = n_iterations
        self.weights = None
        self.bias = None

    def fit(self, X, y):
        n_samples, n_features = X.shape
        # Инициализируем параметры
        self.weights = np.zeros(n_features)
        self.bias = 0

        # Градиентный спуск
        for _ in range(self.n_iters):
            # Предсказание модели
            y_predicted = self._predict_formula(X)

            # Расчет градиентов
            dw = (1 / n_samples) * np.dot(X.T, (y_predicted - y))
            db = (1 / n_samples) * np.sum(y_predicted - y)

            # Обновление весов
            self.weights -= self.lr * dw
            self.bias -= self.lr * db

    def _predict_formula(self, X):
        raise NotImplementedError()

    def predict(self, X):
        raise NotImplementedError()

# Линейная регрессия
class MyLinearRegression(MyBaseRegression):
    def _predict_formula(self, X):
        return np.dot(X, self.weights) + self.bias

    def predict(self, X):
        return self._predict_formula(X)

# Логистическая регрессия
class MyLogisticRegression(MyBaseRegression):
    def _sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def _predict_formula(self, X):
        linear_model = np.dot(X, self.weights) + self.bias
        return self._sigmoid(linear_model)

    def predict(self, X):
        y_predicted = self._predict_formula(X)
        # Возвращаем 0 или 1 в зависимости от порога 0.5
        y_predicted_cls = [1 if i > 0.5 else 0 for i in y_predicted]
        return np.array(y_predicted_cls)


X_clf_train_np = X_clf_train.to_numpy()
y_clf_train_np = y_clf_train.to_numpy()
X_clf_test_np = X_clf_test.to_numpy()

X_reg_train_np = X_reg_train.to_numpy()
y_reg_train_np = y_reg_train.to_numpy()
X_reg_test_np = X_reg_test.to_numpy()

In [14]:
# КЛАССИФИКАЦИЯ (сравнение с бейзлайном)
my_log_reg = MyLogisticRegression(learning_rate=0.01, n_iterations=1000)
my_log_reg.fit(X_clf_train_np, y_clf_train_np)
my_y_clf_pred = my_log_reg.predict(X_clf_test_np)

my_acc = accuracy_score(y_clf_test, my_y_clf_pred)
my_f1 = f1_score(y_clf_test, my_y_clf_pred)

print("Своя реализация vs Sklearn (Бейзлайн)")
print("Классификация:")
print(f"Accuracy: {acc_baseline_lr:.4f} (sklearn) -> {my_acc:.4f} (своя)")
print(f"F1-score: {f1_baseline_lr:.4f} (sklearn) -> {my_f1:.4f} (своя)")
print()


# РЕГРЕССИЯ (сравнение с бейзлайном)
my_lin_reg = MyLinearRegression(learning_rate=0.0001, n_iterations=5000)
my_lin_reg.fit(X_reg_train_np, y_reg_train_np)
my_y_reg_pred = my_lin_reg.predict(X_reg_test_np)

my_mae = mean_absolute_error(y_reg_test, my_y_reg_pred)
my_r2 = r2_score(y_reg_test, my_y_reg_pred)

print("Регрессия:")
print(f"MAE: {mae_baseline_lr:.2f} (sklearn) -> {my_mae:.2f} (своя)")
print(f"R²: {r2_baseline_lr:.4f} (sklearn) -> {my_r2:.4f} (своя)")

Своя реализация vs Sklearn (Бейзлайн)
Классификация:
Accuracy: 0.9855 (sklearn) -> 0.9636 (своя)
F1-score: 0.9843 (sklearn) -> 0.9597 (своя)

Регрессия:
MAE: 9181.31 (sklearn) -> 9325.72 (своя)
R²: 0.1549 (sklearn) -> 0.1382 (своя)


**Выводы по своей реализации (сравнение с бейзлайном)**

Моя реализация логистической регрессии на немасштабированных данных показала результат хуже, чем бейзлайн `sklearn`. Это ожидаемое поведение, так как градиентный спуск очень чувствителен к масштабу признаков, и без их нормализации сходимость затруднена. `Sklearn` использует более сложные и устойчивые численные методы, что позволяет ему работать лучше "из коробки".

Моя реализация линейной регрессии на бейзлайне показывает результаты, сопоставимые с бейзлайном `sklearn`. Это говорит о корректной работе градиентного спуска. Однако, потребовался подбор `learning_rate` и количества итераций, чтобы модель сошлась. `sklearn` делает это эффективнее под капотом.

In [15]:
# КЛАССИФИКАЦИЯ (с улучшениями)

# 1. Берем обработанные данные из пайплайна
scaler_clf = grid_search_clf_lr.best_estimator_.named_steps['scaler']
X_clf_train_scaled = scaler_clf.transform(X_clf_train)
X_clf_test_scaled = scaler_clf.transform(X_clf_test)

# 2. Обучаем нашу модель
my_log_reg_imp = MyLogisticRegression(learning_rate=0.1, n_iterations=1000)
my_log_reg_imp.fit(X_clf_train_scaled, y_clf_train_np)
my_y_clf_pred_imp = my_log_reg_imp.predict(X_clf_test_scaled)

# 3. Оценка
my_acc_imp_lr = accuracy_score(y_clf_test, my_y_clf_pred_imp)
my_f1_imp_lr = f1_score(y_clf_test, my_y_clf_pred_imp)


# РЕГРЕССИЯ (с улучшениями)

# 1. Берем обработанные данные из пайплайна
preprocessor_reg = grid_search_reg_lr.best_estimator_.named_steps['preprocessor']
X_reg_train_processed = preprocessor_reg.transform(X_reg_full_train)
X_reg_test_processed = preprocessor_reg.transform(X_reg_full_test)

# 2. Обучаем нашу модель
my_lin_reg_imp = MyLinearRegression(learning_rate=0.1, n_iterations=1000)
my_lin_reg_imp.fit(X_reg_train_processed, y_reg_full_train.to_numpy())
my_y_reg_pred_imp = my_lin_reg_imp.predict(X_reg_test_processed)

# 3. Оценка
my_mae_imp_lr = mean_absolute_error(y_reg_full_test, my_y_reg_pred_imp)
my_r2_imp_lr = r2_score(y_reg_full_test, my_y_reg_pred_imp)


# ИТОГОВОЕ СРАВНЕНИЕ

print("Своя реализация vs Sklearn (Улучшенные)")
print("Классификация:")
print(f"Accuracy: {acc_improved_lr:.4f} (sklearn) -> {my_acc_imp_lr:.4f} (своя)")
print(f"F1-score: {f1_improved_lr:.4f} (sklearn) -> {my_f1_imp_lr:.4f} (своя)")
print("\nРегрессия:")
print(f"MAE: {mae_improved_lr:.2f} (sklearn) -> {my_mae_imp_lr:.2f} (своя)")
print(f"R²: {r2_improved_lr:.4f} (sklearn) -> {my_r2_imp_lr:.4f} (своя)")

Своя реализация vs Sklearn (Улучшенные)
Классификация:
Accuracy: 0.9855 (sklearn) -> 0.9782 (своя)
F1-score: 0.9843 (sklearn) -> 0.9766 (своя)

Регрессия:
MAE: 4186.91 (sklearn) -> 4181.19 (своя)
R²: 0.7834 (sklearn) -> 0.7836 (своя)


### Выводы по своей реализации (сравнение с улучшенной моделью)

После применения тех же техник предобработки, моя реализация логистической и линейной регрессии показала результаты, очень близкие к улучшенным моделям из `sklearn`. Расхождения с `sklearn` могут быть связаны с тем, что в нем используются более продвинутые и быстрые методы оптимизации, а также в моей реализации отсутствует регуляризация.

**Общий вывод по лабораторной работе №2:**
1.  Линейные модели являются очень мощным и быстрым бейзлайном для многих задач. На "чистых", линейно-разделимых данных они показывают превосходное качество "из коробки".
2.  Как и метрические алгоритмы, они сильно выигрывают от правильной предобработки данных, особенно от масштабирования и кодирования категориальных признаков.
3.  Реализация градиентного спуска с нуля позволила понять фундаментальный принцип обучения большинства моделей в машинном обучении — итеративное уменьшение функции потерь.